In [1]:
from experiments_transformermodels import * 

In [ ]:
# Start a very small run
n_samples_train = 10000
n_samples_val = 1000
toy_model_size = 512
n_epoch = 10
w_load_balancing = 0.01

for i in range(1):
    # Train, save model, evaluate
    val_results = run_experiment_qwen3_moe(n_samples_train=n_samples_train,
                             n_samples_val=n_samples_val,
                             hidden_size=toy_model_size,
                             intermediate_size=toy_model_size,
                             n_epoch=n_epoch,
                             w_load_balancing=w_load_balancing,
                             moe_intermediate_size=toy_model_size,
                                          enable_wandb=True)
    
    # Increment load balancing weight
    w_load_balancing += 0.01

Streaming HuggingFaceFW/fineweb-edu (sample-10BT) (samples: 10000)...


Resolving data files:   0%|          | 0/2410 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/2410 [00:00<?, ?it/s]

wandb: [wandb.login()] Loaded credentials for https://api.wandb.ai from /home/dylan/.netrc.


Model Parameters: 243,450,368


wandb: Currently logged in as: dceveringham (dceveringham-technical-university-of-berlin) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/home/dylan/projects/moe-experiments/experiments_transformermodels.py:131: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 151643, 'pad_token_id': 151643}.
/home/dylan/.local/lib/python3.10/site-packages/datasets/formatting/torch_formatter.py:222: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(value, **{**default_dtype, **self.torch_tensor_kwargs})
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
`use_cache=True` is incompatible with gradi

Step,Training Loss
100,8.517400
200,7.562700
300,7.347200
400,7.130200
500,7.026200
600,6.920500
700,6.836600
800,6.726200
900,6.765400


In [ ]:
for task, metrics in val_results["results"].items():
    print(f"{task}: {metrics}")